<a href="https://colab.research.google.com/github/rikharigaurav/Pytorch/blob/main/pytorch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as f
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms


In [17]:
#Create CNN
class CNN(nn.Module):
  def __init__(self, in_channels = 1, num_classes = 10):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1))
    self.pool = nn.MaxPool2d(kernel_size = (2, 2), stride = (2, 2))
    self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1))
    self.f1 = nn.Linear(16*7*7, num_classes)

  def forward(self, x):
    x = f.relu(self.conv1(x))
    x = self.pool(x)
    x = f.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.f1(x)

    return x


In [18]:
#Set Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
#Hyperparameters
in_channel = 1
num_classes = 10
learning_rate = 0.001
batch_size = 32
num_epochs = 2

In [20]:
#Load Data
train_dataset = datasets.MNIST(root = 'dataset/', train = True, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle =True)
test_dataset = datasets.MNIST(root = 'dataset/', train = True, transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle =True)

In [21]:
#Initialize network
model = CNN().to(device)

In [22]:
#Loss and optimizer
Loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [23]:
# Train Network
for epoch in range(num_epochs):
  for batch_idx, (data, target) in enumerate(train_loader):
    data = data.to(device = device)
    target = target.to(device = device)

    #forward
    predict = model(data)
    loss = Loss(predict, target)

    #backward
    optimizer.zero_grad()
    loss.backward()

    #gradient descent or adam step
    optimizer.step()

# Check accuracy on training & test to see how good our model

def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")
  correct = 0
  samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device = device)
      y = y.to(device = device)

      predict = model(x)
      _, prediction = predict.max(1)
      correct += (prediction == y).sum()
      samples += prediction.size(0)

    print(f"Got {correct}/{samples} with accuracy {float(correct) / float(samples) * 100:.2f}")

    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 58765/60000 with accuracy 97.94
Checking accuracy on training data
Got 58765/60000 with accuracy 97.94
